# Retrieval Augmented Generation (RAG) 애플리케이션 구축: Part 2 - Chatbot Agent 

### 사용자와의 상호작용을 기억하는 메모리 기능과 다단계 검색(multi-step retrieval)을 통합한 RAG 애플리케이션을 구축

- **Part 1**: RAG 개념을 소개하고, 최소한의 구현 방법을 설명합니다.  
- **Part 2** 기존 구현을 확장하여 대화형 상호작용과 다단계 검색 프로세스를 처리할 수 있도록 ReAct agent를 생성합니다.

In [ ]:
# model = init_chat_model("gemini-2.5-flash", model_provider="google_genai")
# 사용할 임베딩 모델의 이름을 지정

### **문서 불러오기 (Loading Documents)**

In [ ]:
# 주요 콘텐츠 태그만 필터링 (제목, 본문, 코드 등)
# WebBaseLoader 사용: requests_kwargs로 User-Agent 설정
# 문서 로드
# 결과 확인
# 불러온 문서를 설정한 기준에 따라 청크로 분할
# 분할된 청크(서브 문서)의 개수 출력

### 검색 단계를 **도구(tool)** 로 전환

- `@tool` 함수는 다음과 같이 `OpenAI의 function-calling` 구조로 함수들을 선언합니다.
```
{
  "name": "retrieve",
  "description": "주어진 쿼리와 관련된 정보를 검색합니다.",
  "parameters": {
    "type": "object",
    "properties": {
      "query": {
        "type": "string",
        "description": "검색할 텍스트 쿼리입니다. llm 이 생성해 줍니다."
      }
    },
    "required": ["query"]
  }
}
```

In [ ]:
# 검색(retrieve) 함수를 도구로 정의
# - response_format="content_and_artifact"로 지정하면 
# 반드시 결과를 직렬화된 content와 artifact로 반환 - Langchain이 format check
def retrieve(query: str):
    # 벡터 스토어에서 유사도 검색 수행. 가장 유사한 두 개의 문서를 반환.
    # 검색된 문서를 문자열로 직렬화
    # 직렬화된 결과(content)와 원본 문서 목록(artifact) 반환.

In [ ]:
# MemorySaver: 워크플로우의 상태를 메모리에 저장

------------------------------
에이전트가 수행하는 과정은 다음과 같습니다:  

1. **작업 분해(Task Decomposition)의 표준 방법을 검색하기 위한 쿼리를 생성합니다.**  
2. **첫 번째 답변을 받은 후, 해당 방법의 일반적인 확장을 검색하는 두 번째 쿼리를 생성합니다.**  
3. **필요한 모든 문맥을 확보한 후, 최종적으로 질문에 대한 답변을 생성합니다.**  